### Importando librerías

In [1]:
# Importando librería Geopandas
import geopandas as gpd

# Instalar la librería mapclassify
#!pip install mapclassify

### Importando datos

In [2]:
# Importando 'comercios.geojson' y creando un geodataframe
comercios_puntos = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/data/comercios.geojson')

In [ ]:
# Explorando datos en el mapa
comercios_puntos.explore()

### Unión de tablas

In [4]:
# Importando 'comercios_datos.csv' y creando un geodataframe
comercios_datos = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/data/comercios_datos.csv')

In [ ]:
# Explorando los datos del geodataframe
comercios_datos.head()

In [ ]:
# Haciendo la unión de tablas
comercios_puntos.merge(comercios_datos, left_on='id', right_on='id')

In [7]:
# Almacenando el resultado en una variable
comercios = comercios_puntos.merge(comercios_datos, left_on='id', right_on='id')

In [8]:
# Modificando el nombre de los campos. El orden de los campos es indistinto
comercios = comercios.rename(columns={'id':'id',
                                        'geometry_x':'geometry',
                                        'nombre': 'Nombre',
                                        'domicilio':'Dirección',
                                        'nombre':'Comercio',
                                        'actividad':'Actividad',
                                        'descrubro':'Descripción',
                                        'categoria':'Categoría'})

In [ ]:
# Explorando 'comercios' luego de renombrar los campos
comercios.head()

In [ ]:
# Consultar el tipo de objeto que es 'comercios'
type(comercios)

In [11]:
# Creando un nuevo geodataframe a partir de los datos de geometría del objeto 'comercios' en una variable auxiliar
aux = gpd.GeoDataFrame(comercios, geometry='geometry')

In [ ]:
# Verificando que 'aux' sea de tipo geodataframe
type(aux)

In [13]:
# Redefiniendo la variable 'comercios' de acuerdo al valor de 'aux'
comercios = aux

Trabajando con CRS

In [ ]:
# Consultando el CRS del geodataframe 'comercios'
comercios.crs

In [15]:
# Reproyectando a PosGAr 2007 faja 5 (EPSG:5347)
comercios_5347 = comercios.to_crs(epsg=5347)

In [ ]:
comercios_5347.crs

In [ ]:
comercios_5347.explore()

### Creando y personalizando el mapa



Importando la librería Folium

In [18]:
import folium # Importando la librería Folium para acceder a más opciones de configuración sobre el mapa

Creando geodataframe de la capa 'Circuitos electorales'

In [19]:
# Importar una nueva capa
circuitos = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/data/circuitos_electorales.shp', encoding='utf8') # Cuando llamo un shapefile, referenciar al archivo de extensión .shp

In [ ]:
# Exploramos los datos del geodataframe 'circuitos'
circuitos.head()

In [ ]:
# Creando el objeto 'mapa' a partir de explorar los datos del geodataframe 'circuitos'
mapa = circuitos.explore(
                        column='Nombre',                # Defino la columna por la cual se van a representar las geometrías
                        name="Circuitos electorales",   # Nombre representativo de la capa que estamos explorando
                        legend=False                    # Desactivo la leyenda para que luego no se superponga con otras
                    )
mapa

Personalizando la visualización del geodataframe 'comercios_5347'

In [ ]:
comercios_5347.explore(
                column='Categoría',                     # Define qué campo vamos a utilizar para categorizar los puntos
                m=mapa,                                 # Agrega 'comercios_5347' al objeto 'mapa'
                cmap=['green', '#2C2C2C', 'blue'],      # Define el color de los puntos por cada uno de los valores de 'Categoría'
                marker_kwds=dict(radius=5, fill=True),  # Define el radio del punto y si debería incluir relleno
                style_kwds=dict(                        # Agrega parámetros avanzados para la configuración de los puntos
                    stroke=True,                          # Muestra el borde del marcador
                     color='#232323',                     # Define el color del borde
                     weight=0.7,                          # Define el ancho del borde
                     opacity=1,                           # Define la opacidad del borde del marcador
                     fillOpacity=0.85                     # Define la opacidad del relleno del marcador
                    ),
                name="Comercios",                       # Nombre representativo de la capa que estamos explorando
                tooltip="Categoría",                    # Muestra el valor de 'Categoría' en el tooltip al pasar el cursor encima
                popup=True,                             # Muestra las variables y sus valores al hacer click sobre cada punto
            )

Agregar una nueva capa de puntos al mapa

In [23]:
# Importar una tercera capa
centros_salud = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/data/centros_salud.shp', encoding='utf8')

In [ ]:
centros_salud.explore(
    m=mapa,                                     # Agrega la capa al objeto de nombre 'mapa'
    color='red',                                # Le asigna el color 'red' a los puntos
    marker_kwds=dict(radius=5, fill=True),      # Define el radio del punto y si debería incluir relleno
    name="Centros de salud",                    # Nombre representativo de la capa que estamos explorando
    tooltip="Nombre",                           # Muestra el valor de 'Nombre' en el tooltip al pasar el cursor encima
    )

In [ ]:
# Agregando un control de capas al mapa
folium.LayerControl().add_to(mapa)
mapa

### Exportando en HTML

In [26]:
mapa.save(r"mapa.html")